In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Input, BatchNormalization, Add, AveragePooling2D, MaxPooling2D, ZeroPadding2D, ReLU, Dense, Flatten
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
!wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
!unzip -qq cell_images.zip

--2023-02-13 23:25:44--  https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
Resolving data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)... 13.249.85.51, 13.249.85.53, 13.249.85.110, ...
Connecting to data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)|13.249.85.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353452851 (337M) [application/zip]
Saving to: ‘cell_images.zip’

cell_images.zip     100%[===================>] 337.08M  40.3MB/s    in 8.5s    

2023-02-13 23:25:53 (39.6 MB/s) - ‘cell_images.zip’ saved [353452851/353452851]



In [3]:
datagen = ImageDataGenerator(rescale=1./255, # scale pixel to (0, 1) value
                            validation_split=0.2) # set validation split

In [4]:
data_dir = '/content/cell_images'

In [14]:
image_size = (32, 32)
batch_size = 128
epochs = 10

In [15]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training') # set as training data

Found 22048 images belonging to 2 classes.


In [16]:
validation_generator = datagen.flow_from_directory(
    data_dir, # same directory as training data
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') # set as validation data

Found 5510 images belonging to 2 classes.


In [17]:
image, label = next(train_generator)

In [18]:
image.shape

(128, 32, 32, 3)

In [19]:
train_generator.class_indices # Label map 

{'Parasitized': 0, 'Uninfected': 1}

In [20]:
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [21]:
def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

In [29]:
def ResNet34(shape = (32, 32, 3), classes = 1):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

In [30]:
model = ResNet34(shape=(*image_size, 3), classes=1)

In [31]:
model.summary()

Model: "ResNet34"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 38, 38, 3)   0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_36 (Conv2D)             (None, 19, 19, 64)   9472        ['zero_padding2d_1[0][0]']       
                                                                                                  
 batch_normalization_33 (BatchN  (None, 19, 19, 64)  256         ['conv2d_36[0][0]']       

In [32]:
# If the val loss did not decrease in 3 times, model will stop training and restore best weights
early_stopping = EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)

In [33]:
model.compile(optimizer='adam', loss='bce', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=epochs, callbacks=[early_stopping])

Epoch 1/10
173/173 [==============================] - 3664s 21s/step - loss: 0.5698 - accuracy: 0.5000 - val_loss: 3.9565 - val_accuracy: 0.5000
Epoch 2/10
 27/173 [===>..........................] - ETA: 41:37 - loss: 0.3814 - accuracy: 0.5136

In [ ]:
pd.DataFrame(history.history).plot()
plt.grid(True)
plt.xlabel("epochs")
plt.show()